### Практическое занятие: Асинхронная обработка и задачи в фоновом режиме

---

#### Цель занятия:
1. Изучить использование горутин в Go для выполнения асинхронных операций.
2. Реализовать фоновые задачи в REST API.
3. Научиться управлять состоянием задач в реальном времени.

---

### Теоретическая часть:

**1. Асинхронная обработка в Go:**
- Go использует лёгкие потоки — **горутины**, которые позволяют выполнять задачи параллельно.
- Горутины создаются с помощью ключевого слова `go`:
  ```go
  go func() {
      fmt.Println("Асинхронная задача выполняется!")
  }()
  ```
- Для синхронизации используется `sync.WaitGroup`, каналы (`chan`) или другие механизмы.

**2. Фоновые задачи:**
- Задачи в фоновом режиме полезны для длительных операций (например, экспорта данных, отправки писем).
- Часто используют очереди задач или внешние системы для обработки, но в простых случаях можно обойтись горутинами.

**3. Хранение состояния задач:**
- Для управления состоянием задач (выполняется, завершена, произошла ошибка) можно использовать карты (`map`) или специализированные структуры.

---

### Практическая часть:

#### Шаг 1: Подготовка проекта
1. Установите библиотеку Gin:
   ```bash
   go get -u github.com/gin-gonic/gin
   ```

2. Создайте структуру проекта:
   ```
   - main.go
   - tasks/
       - tasks.go
   ```

---

#### Шаг 2: Реализация фоновой задачи экспорта данных

**1. Создание файла `tasks/tasks.go`:**

```go
package tasks

import (
    "encoding/json"
    "os"
    "sync"
    "time"
)

type Task struct {
    ID       string `json:"id"`
    Status   string `json:"status"`   // pending, in_progress, done, error
    Filename string `json:"filename"`
}

var tasks = make(map[string]*Task)
var mutex = &sync.Mutex{}

func CreateTask() string {
    taskID := generateID()
    task := &Task{
        ID:     taskID,
        Status: "pending",
    }
    mutex.Lock()
    tasks[taskID] = task
    mutex.Unlock()
    return taskID
}

func GetTask(taskID string) *Task {
    mutex.Lock()
    defer mutex.Unlock()
    return tasks[taskID]
}

func RunTask(taskID string) {
    task := GetTask(taskID)
    if task == nil {
        return
    }

    task.Status = "in_progress"
    filename := "export_" + taskID + ".json"
    task.Filename = filename

    // Эмуляция длительного процесса
    time.Sleep(5 * time.Second)

    // Запись данных в файл
    file, err := os.Create(filename)
    if err != nil {
        task.Status = "error"
        return
    }
    defer file.Close()

    data := map[string]string{"message": "Data exported successfully"}
    json.NewEncoder(file).Encode(data)

    task.Status = "done"
}

func generateID() string {
    return time.Now().Format("20060102150405")
}
```

**2. Создание маршрутов в `main.go`:**

```go
package main

import (
    "github.com/gin-gonic/gin"
    "tasks"
)

func main() {
    router := gin.Default()

    // Создание задачи
    router.POST("/tasks", func(c *gin.Context) {
        taskID := tasks.CreateTask()
        go tasks.RunTask(taskID)
        c.JSON(201, gin.H{"task_id": taskID})
    })

    // Получение статуса задачи
    router.GET("/tasks/:id", func(c *gin.Context) {
        taskID := c.Param("id")
        task := tasks.GetTask(taskID)
        if task == nil {
            c.JSON(404, gin.H{"error": "Task not found"})
            return
        }
        c.JSON(200, task)
    })

    router.Run(":8080")
}
```

---

### Задания для студентов:

1. **Основное задание:**

    - Внедрите рассмотренную выше технологию в Ваш основной проект.
    - Добавьте логирование действий в консоль для каждого этапа выполнения задач

2. **Дополнительное задание:**

    - Добавьте возможность отмены задачи.
    - Реализуйте ограничение на количество одновременно выполняющихся задач (например, не более 5).

3. **Тестирование:**

    - Проверьте работу приложения при создании нескольких задач подряд.
    - Убедитесь, что задача завершает выполнение через 5 секунд.

---
